## BRONZE TO SILVER LAYER

### GOLD LAYER - PROCESS HOLDING RECORDS HISTORY


In [24]:
df = pd.read_csv(global_path.holdings_gold_file_path)
# Retain only rows with maximum datetime for each 'date' and 'stock_name'
df = df.merge(df.groupby(["exchange","segment","stock_name"])["datetime"].max().reset_index()
, on=["datetime","exchange","segment","stock_name"])
df = df[df["holding_quantity"]!= 0].reset_index(drop=True)
# # Add a 'date' column by extracting the date part from 'datetime'
# df["date"] = df["datetime"].dt.date

# Sort DataFrame by 'segment', 'stock_name', and 'datetime'
df = df.sort_values(by=["segment", "stock_name", "datetime"])
df



,datetime,segment,exchange,stock_name,holding_quantity,holding_price_avg,holding_amount
0,2024-06-04 12:19:21,EQ,NSE,IRCTC,3.00,896.20,2688.60
1,2024-06-13 13:58:03,EQ,NSE,KPITTECH,2.00,1500.00,3000.00
2,2022-05-11 11:50:35,EQ,NSE,TATAMOTORS,100.00,261.00,26100.40
3,2022-08-10 09:30:24,EQ,NSE,TATAPOWER,150.00,147.49,22124.00
4,2024-06-20 10:02:14,FO,FON,NIFTY-CE-23450-20JUN2024,25.00,70.00,1750.00
5,2024-06-21 13:56:13,FO,FON,NIFTY-CE-23500-27JUN2024,-25.00,151.00,-3775.00
6,2024-06-20 12:28:31,FO,FON,NIFTY-CE-23650-20JUN2024,75.00,14.75,1106.25
7,2024-07-02 14:54:13,FO,FON,NIFTY-CE-24000-04JUL2024,75.00,181.55,13616.25
8,2024-07-02 14:54:13,FO,FON,NIFTY-CE-24000-04JUL2024,50.00,181.55,9077.50
9,2024-07-01 13:45:26,FO,FON,NIFTY-CE-24050-04JUL2024,75.00,182.85,13713.75


In [11]:
# Import necessary libraries and utility functions
import pandas as pd
from common_utilities import global_path, logger
from datetime import datetime

### Data Processing

# - Load and Filter trade history data from the Gold layer.

# Load trade history from CSV into DataFrame
df = pd.read_csv(global_path.holdings_gold_file_path)
logger.info(
    f"Loaded GOLD Layer trade history data from: {global_path.tradehistory_gold_file_path}"
)

# # Filter for specific segments
# df = df[df["segment"].isin(["EQ", "MF"])]

# Convert 'datetime' column to datetime objects
df["datetime"] = pd.to_datetime(df["datetime"])

# Add a 'date' column by extracting the date part from 'datetime'
df["date"] = df["datetime"].dt.date

# Sort DataFrame by 'segment', 'stock_name', and 'datetime'
df = df.sort_values(by=["segment", "stock_name", "datetime"])


### Data Processing

# - Process data to include all dates up to today.
# - Merge with stock price data and calculate current values.
# - Save the processed data to the Gold layer.

# Get maximum 'datetime' for each 'date' and 'stock_name' combination

max_datetime_df = (
    df.groupby(["date", "stock_name"])["datetime"].max().reset_index()
)

# Retain only rows with maximum datetime for each 'date' and 'stock_name'

df = df.merge(max_datetime_df, on=["date", "stock_name", "datetime"])

# Sort the DataFrame by 'segment', 'stock_name', and 'date'

df = df.sort_values(by=["segment", "stock_name", "date"]).reset_index(drop=True)

# Process each stock name separately

stock_names = df["stock_name"].unique()

result = []

for stock_name in stock_names:
    stock_data = df[df["stock_name"] == stock_name].copy()

    # Set 'date' as index and drop duplicate indices, keeping the first occurrence
    stock_data = stock_data.set_index("date")
    stock_data = stock_data[~stock_data.index.duplicated(keep="first")]

    # Ensure there are no duplicate dates before reindexing
    date_range = pd.date_range(
        start=stock_data.index.min(),
        end=datetime.today().date(),
        freq="D",
    )

    # Reindex to include all dates and forward fill missing values
    stock_data = stock_data.reindex(date_range, method="ffill")

    # Reset index to bring 'date' back as a column
    stock_data = stock_data.reset_index().rename(columns={"index": "date"})
    result.append(stock_data)

# Combine processed DataFrames into one

df = pd.concat(result, ignore_index=True)

# Load stock prices and merge with the main DataFrame

df_StockPrice = pd.read_csv(global_path.stockprice_silver_file_path)

df_StockPrice = df_StockPrice[["date", "stock_name", "close"]]

df_StockPrice["date"] = pd.to_datetime(df_StockPrice["date"])

logger.info(
    f"Loaded SILVER Layer stock price data from: {global_path.stockprice_silver_file_path}"
)

# Merge stock price data

df = pd.merge(df, df_StockPrice, on=["date", "stock_name"], how="left")

# Rename columns for clarity and calculate current value

df = df.rename(
    columns={
        "close": "ltp",
    }
)

df["current_value"] = df["ltp"] * df["holding_quantity"]

# Calculate PnL and percentage

df["pnl_amount"] = df["current_value"] - df["holding_amount"]

df["pnl_percentage"] = (df["pnl_amount"] / df["holding_amount"]) * 100

# Filter out rows with zero 'holding_quantity"

df = df[(df["holding_amount"] != 0) & (df["current_value"] != 0)]

# Round the values to two decimal places

df = df.round(2)

# Final sorting and column selection

df = df.sort_values(by=["segment", "stock_name", "date"]).reset_index(drop=True)

# Save the final DataFrame to a CSV file

df = df[
    [
        "date",
        "segment",
        "stock_name",
        "holding_quantity",
        "holding_price_avg",
        "holding_amount",
        "ltp",
        "current_value",
        "pnl_amount",
        "pnl_percentage",
    ]
]
df.to_csv(global_path.holdings_gold_file_path_v2, index=None)
logger.info("GOLD Layer CSV file for Holdings successfully created at:")
logger.info(global_path.holdings_gold_file_path_v2.resolve())
# Display DataFrame information and print success message
df.info()

2024-08-05T19:04:43Z - INFO - Loaded GOLD Layer trade history data from: C:\Users\prashant.tripathi\Code\Upstox\DATA\GOLD\TradeHistory\TradeHistory_data.csv
2024-08-05T19:04:43Z - INFO - Loaded SILVER Layer stock price data from: C:\Users\prashant.tripathi\Code\Upstox\DATA\SILVER\StockPrice\StockPrice_data.csv
2024-08-05T19:04:43Z - INFO - GOLD Layer CSV file for Holdings successfully created at:
2024-08-05T19:04:43Z - INFO - C:\Users\prashant.tripathi\Code\Upstox\DATA\GOLD\Holdings\Holdings_datav2.csv


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6336 entries, 0 to 6335
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   date               6336 non-null   datetime64[ns]
 1   segment            6336 non-null   object        
 2   stock_name         6336 non-null   object        
 3   holding_quantity   6336 non-null   float64       
 4   holding_price_avg  6336 non-null   float64       
 5   holding_amount     6336 non-null   float64       
 6   ltp                3827 non-null   float64       
 7   current_value      3827 non-null   float64       
 8   pnl_amount         3827 non-null   float64       
 9   pnl_percentage     3827 non-null   float64       
dtypes: datetime64[ns](1), float64(7), object(2)
memory usage: 495.1+ KB
